In [88]:
import numpy as np
import numpy as np
from scipy.sparse import coo_matrix
import fast_matrix_market as fmm
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn import tree

params = [[150, 0.95], [200, 0.95], [300, 0.85], [400, 0.6], [500, 0.6], [1000, 0.4], [1500, 0.4]]

In [63]:
def get_graph_lists(graph): # Преобразование графа в список и получение списков смежности и степеней для вершин
    graph = list(graph)
    for i in range(len(graph)):
        graph[i] = list(graph[i])

    adj_list = {}
    degrees = {}
    for i in range(len(graph)):
        temp = []
        degree = 0
        for j in range(len(graph)):
            if graph[i][j] == 1.0:
                degree += 1
                temp.append(j)
        adj_list[i] = temp
        degrees[i] = degree

    degrees = {k: v for k, v in sorted(degrees.items(), key=lambda item: item[1], reverse = True)}
    return degrees, adj_list

In [64]:
def greedy_search(degrees, adj_list, graph): #Жадный поиск
    candidates = list(degrees.keys())
    clique = [candidates[0]]

    while True:
        candidate_index = 0 
        to_drop = []

        #print(clique, candidates)

        for candidate in candidates: #Перестроение списка кандидатов
            for vertex in clique:
                if graph[vertex][candidate] == 0:
                    to_drop.append(candidate_index)
                    break
                
            candidate_index += 1

        candidates = [i for j, i in enumerate(candidates) if j not in to_drop]
        
        if len(candidates) == 0: #Если кандидаты ещё есть - добавление в клику лучшего, иначе завершение программы
            break
        clique.append(candidates[0])

    return clique

In [65]:
def greedy_coloring(adj_list): #Жадное окрашивание
    vertices = sorted(list(adj_list.keys()))
    colour_graph = {}

    for vertex in vertices:
        unused_colours = len(vertices) * [True] #Список неиспользованных цветов

        for neighbor in adj_list[vertex]: #Отмечаются все занятые соседями цвета
            if neighbor in colour_graph:
                colour = colour_graph[neighbor] 
                unused_colours[colour] = False

        for colour, unused in enumerate(unused_colours): #Вершине присваивается первый незанятый цвет
            if unused:
                colour_graph[vertex] = colour
                break

    return colour_graph

In [66]:
def get_graph_params(graph):
    degrees, adj_list = get_graph_lists(graph)
    greedy_clique = greedy_search(degrees, adj_list, graph)
    colored = greedy_coloring(adj_list)

    gs_res = len(greedy_clique)

    gc_res = len(np.unique(list(colored.values())))

    vertex_num = len(graph)

    edges_num = 0
    degrees_sum = []
    for i in range(vertex_num):
        for j in range(i, vertex_num):
            if graph[i][j] > 0:
                edges_num += 1
                degrees_sum.append(degrees[i] + degrees[j])
    density = edges_num / vertex_num

    degrees_list = list(degrees.values())
    min_deg = min(degrees_list)
    max_deg = max(degrees_list)
    mean_deg = np.mean(degrees_list)
    std_deg = np.std(degrees_list)

    deg_sum_max = max(degrees_sum)
    deg_sum_min = min(degrees_sum)
    deg_sum_mean = np.mean(degrees_sum)
    deg_sum_std = np.std(degrees_sum)
    
    graph_params = []
    graph_params.append(vertex_num)
    graph_params.append(edges_num)
    graph_params.append(density)
    graph_params.append(min_deg)
    graph_params.append(max_deg)
    graph_params.append(mean_deg)
    graph_params.append(std_deg)
    graph_params.append(deg_sum_min)
    graph_params.append(deg_sum_max)
    graph_params.append(deg_sum_mean)
    graph_params.append(deg_sum_std)
    graph_params.append(gs_res)
    graph_params.append(gc_res)

    return graph_params

In [67]:
features = []
answers = []

for i in range(len(params)):
    limit = int(params[i][1] // 0.05)
    for j in range(limit + 1):
        possibility = min((j + 1) * 0.05, params[i][1])
        filename = "graphI" + str(params[i][0]) + "I" + str(int(possibility * 100)) + "I"
        serie_results = list(np.genfromtxt("resultsI" + filename + "9.csv", delimiter = " ,"))
        answers += serie_results
        for k in range(10):
            graph_params = list(np.genfromtxt("paramsI" + filename + str(k) + ".csv", delimiter = " ,"))
            features.append(graph_params)

In [68]:
(data, (rows, cols)), shape = fmm.read_coo("brock800-3.mtx")
graph = coo_matrix((data, (rows, cols)), shape).toarray()
graph_params = get_graph_params(graph)

In [89]:
log_regr = LogisticRegression(solver='lbfgs', max_iter=10000, multi_class='ovr')
log_regr.fit(features, answers)
lin_regr = LinearRegression()
lin_regr.fit(features, answers)
svc = svm.SVC()
svc.fit(features, answers)
bayes = GaussianNB()
bayes.fit(features, answers)
trees = tree.DecisionTreeClassifier()
trees.fit(features, answers)

DecisionTreeClassifier()

In [92]:
log_res = int(log_regr.predict(np.array(graph_params).reshape(1, -1)))
lin_res = int(lin_regr.predict(np.array(graph_params).reshape(1, -1)))
svc_res = int(svc.predict(np.array(graph_params).reshape(1, -1)))
bayes_res = int(bayes.predict(np.array(graph_params).reshape(1, -1)))
trees_res = int(trees.predict(np.array(graph_params).reshape(1, -1)))

print(log_res, lin_res, svc_res, bayes_res, trees_res)

3 3 4 3 4


C:\Users\Иван\AppData\Local\Temp\ipykernel_7836\698381765.py:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  log_res = int(log_regr.predict(np.array(graph_params).reshape(1, -1)))
C:\Users\Иван\AppData\Local\Temp\ipykernel_7836\698381765.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  lin_res = int(lin_regr.predict(np.array(graph_params).reshape(1, -1)))
C:\Users\Иван\AppData\Local\Temp\ipykernel_7836\698381765.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)